In [2]:
import gc
import os

import numpy as np
import pandas as pd
import random
from sklearn.metrics import roc_auc_score
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset

import pytorch_lightning as pl

from transformers import DebertaV2Config, DebertaV2Model

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TRANSFORMERS_CACHE'] = './cache/'

In [3]:
torch.__version__, torch.cuda.is_available()

('2.0.1+cu118', True)

In [4]:
pl.seed_everything(56, workers=True)

Global seed set to 56


56

In [5]:
columns = ['variantid', 'main_pic_embeddings_resnet_v1', 'name_bert_64']
train_data = pd.read_parquet('./datasets/train_data.parquet', columns=columns).set_index('variantid')
test_data = pd.read_parquet('./datasets/test_data.parquet', columns=columns).set_index('variantid')
train_data

,main_pic_embeddings_resnet_v1,name_bert_64
variantid,,
51195767,"[[0.04603629, 0.18839523, -0.09973055, -0.6636...","[-0.47045058, 0.67237014, 0.48984158, -0.54485..."
53565809,"[[1.1471839, -0.665361, 0.7745614, 0.26716197,...","[-0.6575592, 0.6522429, 0.5426037, -0.54347897..."
56763357,"[[-0.90570974, 1.0296293, 1.0769907, 0.27746, ...","[-0.7384308, 0.70784587, 0.3012653, -0.3583719..."
56961772,"[[0.13133773, -0.5577079, 0.32498044, 0.191717...","[-0.44812852, 0.5283565, 0.28981736, -0.506841..."
61054740,"[[0.21696381, 0.10989461, -0.08012986, 0.69186...","[-0.72692573, 0.75206333, 0.37740713, -0.52502..."
...,...,...
820128810,"[[-1.4492652, -0.80129164, -0.12344764, 0.7194...","[-0.8253241, 0.6785133, 0.53978086, -0.4888316..."
821135769,"[[0.012127608, -0.8534423, 0.5415518, -0.44912...","[-0.7413257, 0.46105132, 0.5639801, -0.5462132..."
822095690,"[[0.4248176, -0.15944786, -0.22844064, 0.42768...","[-0.49261805, 0.56726897, 0.7037877, -0.697246..."


In [6]:
train_pairs = pd.read_parquet('./datasets/train_pairs_w_target.parquet')
test_pairs = pd.read_parquet('./datasets/test_pairs_wo_target.parquet')
train_pairs['target'] = train_pairs['target'].astype(int)
train_pairs

,target,variantid1,variantid2
0,0,51197862,51198054
1,1,53062686,536165289
2,1,53602615,587809782
3,1,53888651,89598677
4,0,56930698,551526166
...,...,...,...
306535,0,817327230,822083612
306536,0,817560551,818069912
306537,0,817854719,817857267
306538,0,820036017,820037019


In [7]:
name_labse_embs = pd.read_parquet('datasets/name_labse_embs.parquet').set_index('variantid')
name_labse_embs

,name_labse_768
variantid,
51195767,"[-0.033874325, 0.03722446, 0.0029757991, 0.068..."
53565809,"[0.015568526, -0.03899538, 0.064447366, 0.0383..."
56763357,"[-0.033072222, -0.04237577, 0.020771954, 0.065..."
56961772,"[0.014727573, -0.025661988, 0.023943473, -0.00..."
61054740,"[0.043145332, -0.052424084, 0.017260496, 0.045..."
...,...
820128810,"[-0.003678058, -0.031628493, 0.0065589263, 0.0..."
821135769,"[-0.06858361, 0.027011767, -0.016400583, -0.02..."
822095690,"[-0.04474233, -0.034224413, 0.026076552, 0.026..."


In [8]:
val_pairs = train_pairs[pd.read_csv('./datasets/val_idx.csv', index_col=0).values].copy()
train_pairs = train_pairs[pd.read_csv('./datasets/train_idx.csv', index_col=0).values].copy()

In [11]:
class Args:
    batch_size = 512
    epochs = 15
    lr = 1e-4
    lr_warmup_epochs = 5
    lr_warmup_decay = 0.01
    lr_min = 1e-5
    
args = Args()

In [12]:
class ItemsDataset(Dataset):
    def __init__(self, pairs, data):
        super().__init__()
        self.pairs = pairs.values
        self.main_pic_embs = data['main_pic_embeddings_resnet_v1']
        self.name_embs = data['name_bert_64']
        self.name_labse_embs = name_labse_embs['name_labse_768']
        self.pairs_len = len(self.pairs)
        
    def __len__(self):
        return self.pairs_len

    def __getitem__(self, idx):
        target, id1, id2 = self.pairs[idx, :]
        return (
            torch.tensor(np.concatenate([self.main_pic_embs[id1][0], self.name_embs[id1], self.name_labse_embs[id1]])),
            torch.tensor(np.concatenate([self.main_pic_embs[id2][0], self.name_embs[id2], self.name_labse_embs[id2]])),
            target
        )

In [13]:
def get_data_loader(pairs, data, batch_size, drop_last, shuffle):
    dataset = ItemsDataset(pairs, data)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        num_workers=0,
        drop_last=drop_last,
        shuffle=shuffle,
        pin_memory=True
    )
    return data_loader

In [14]:
def get_loaders(args):
    train_loader = get_data_loader(
        pairs=train_pairs,
        data=train_data,
        batch_size=args.batch_size,
        drop_last=True,
        shuffle=True
    )
    
    val_loader = get_data_loader(
        pairs=val_pairs,
        data=train_data,
        batch_size=args.batch_size,
        drop_last=False,
        shuffle=False
    )
    return train_loader, val_loader

In [15]:
train_loader, val_loader = get_loaders(args)
len(train_loader), len(val_loader)

(399, 200)

In [16]:
next(iter(val_loader))

[tensor([[-3.2774e-03,  3.2531e-01, -3.3157e-01,  ...,  3.1994e-02,
          -3.3705e-02,  1.7488e-02],
         [ 5.0361e-01,  1.1200e+00,  4.5101e-01,  ..., -3.0027e-03,
          -1.4640e-02,  3.1890e-02],
         [ 6.6419e-01,  1.1968e-01, -4.1827e-01,  ..., -4.8692e-02,
           1.8094e-02, -6.9256e-03],
         ...,
         [ 1.7638e-01,  3.5629e-01,  5.9098e-01,  ..., -1.5599e-02,
          -7.0778e-02, -1.9628e-04],
         [ 3.3006e-01, -3.6003e-01,  2.6317e-01,  ...,  5.3874e-02,
          -2.4707e-02, -3.2029e-03],
         [-1.3787e-01,  2.6778e-01,  6.7039e-01,  ..., -9.7806e-03,
           3.0475e-02,  4.1302e-02]]),
 tensor([[-0.0436,  0.4931, -0.3070,  ...,  0.0513,  0.0016,  0.0265],
         [ 0.4324,  0.8119,  0.3148,  ...,  0.0115, -0.0241,  0.0392],
         [ 0.5154, -0.0099, -0.0911,  ..., -0.0433, -0.0030,  0.0253],
         ...,
         [ 0.1687,  0.4118,  0.5480,  ..., -0.0255, -0.0697, -0.0030],
         [ 0.6076, -0.3946,  0.6024,  ...,  0.0539, -0.0

In [17]:
class Net(pl.LightningModule):
    margin = 0.75
    
    def __init__(self):
        super(Net, self).__init__()

        input_size = 128 + 64 + 768
        output_size = 512
        # self.stabilizer = nn.Linear(input_size, input_size)
        self.bn = nn.BatchNorm1d(input_size)
        
        deberta_cfg = DebertaV2Config(
            hidden_size=input_size,
            num_hidden_layers=1,
            num_attention_heads=1,
            intermediate_size=1024,
        )
        self.deberta = DebertaV2Model(deberta_cfg, ).encoder
        
        features_num = 2 * input_size
        embedding_size = (features_num + output_size) // 2
        self.neck = nn.Sequential(
            nn.BatchNorm1d(features_num),
            nn.Linear(features_num, embedding_size, bias=False),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(embedding_size),
            nn.Linear(embedding_size, embedding_size, bias=False),
            nn.BatchNorm1d(embedding_size),
        )
        
        self.output_layer = nn.Linear(embedding_size, output_size)
        
    def forward(self, x):
        # x = self.stabilizer(x)
        x = self.bn(x)
        x = x.unsqueeze(1)
        attention_mask = torch.ones((x.shape[0], 1), device='cuda')
        last_hidden = self.deberta(x, attention_mask)
        last_hidden = torch.concat([last_hidden[0].mean(1), last_hidden[0].max(1)[0]], -1)
        outputs = self.neck(last_hidden)
        outputs = self.output_layer(outputs)
        return outputs

    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(), lr=args.lr, betas=(0.9, 0.999), weight_decay=0.05
        )
        main_lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=args.epochs - args.lr_warmup_epochs, eta_min=args.lr_min
        )
        warmup_lr_scheduler = torch.optim.lr_scheduler.LinearLR(
            optimizer, start_factor=args.lr_warmup_decay, total_iters=args.lr_warmup_epochs
        )
        lr_scheduler = torch.optim.lr_scheduler.SequentialLR(
            optimizer, schedulers=[warmup_lr_scheduler, main_lr_scheduler], milestones=[args.lr_warmup_epochs]
        )
        return (
            {
                "optimizer": optimizer,
                "lr_scheduler": {
                    "scheduler": lr_scheduler,
                    "interval": "epoch",
                    "frequency": 1,
                    "strict": True,
                },
            },
        )
    
    def training_step(self, batch, batch_idx):
        # self.log('step', batch_idx, logger=True, on_epoch=True)
        x1, x2, labels = batch
        out1 = self.forward(x1)
        out2 = self.forward(x2)
        
        fnorm = torch.norm(out1, p=2, dim=1, keepdim=True)
        out1 = out1.div(fnorm.expand_as(out1))
        fnorm = torch.norm(out2, p=2, dim=1, keepdim=True)
        out2 = out2.div(fnorm.expand_as(out2))
        
        dists = nn.PairwiseDistance()(out1, out2)
        loss = (labels) * torch.pow(dists, 2) + (1 - labels) * torch.pow(torch.clamp(self.margin - dists, min=0.0), 2)
        loss = torch.mean(loss)
        self.log("train_loss", loss, on_step=False, logger=False, on_epoch=True, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):        
        x1, x2, labels = batch
        out1 = self.forward(x1)
        out2 = self.forward(x2)
        
        fnorm = torch.norm(out1, p=2, dim=1, keepdim=True)
        out1 = out1.div(fnorm.expand_as(out1))
        fnorm = torch.norm(out2, p=2, dim=1, keepdim=True)
        out2 = out2.div(fnorm.expand_as(out2))
        
        dists = nn.PairwiseDistance()(out1, out2)
        loss = (labels) * torch.pow(dists, 2) + (1 - labels) * torch.pow(torch.clamp(self.margin - dists, min=0.0), 2)
        loss = torch.mean(loss)
        self.log("val_loss", loss, logger=False, on_epoch=True, prog_bar=True)   
        
        try:
            auc = roc_auc_score(labels.detach().cpu(), 1 - dists.detach().cpu())
        except:
            auc = 0
            
        self.log("val_auc", auc, logger=False, on_epoch=True, prog_bar=True)
        
    def train_dataloader(self):
        return train_loader

    def val_dataloader(self):
        return val_loader
    
    def predict_step(self, batch, batch_idx):
        x1, x2, labels = batch
        out1 = self.forward(x1)
        out2 = self.forward(x2)
        
        fnorm = torch.norm(out1, p=2, dim=1, keepdim=True)
        out1 = out1.div(fnorm.expand_as(out1))
        fnorm = torch.norm(out2, p=2, dim=1, keepdim=True)
        out2 = out2.div(fnorm.expand_as(out2))
        
        dists = nn.PairwiseDistance()(out1, out2)
        return torch.cat([out1, out2, (1 - dists).unsqueeze(-1)], dim=1).detach().cpu()

In [18]:
model = Net()

In [19]:
import torchinfo

torchinfo.summary(model)

Layer (type:depth-idx)                                       Param #
Net                                                          --
├─BatchNorm1d: 1-1                                           1,920
├─DebertaV2Encoder: 1-2                                      --
│    └─ModuleList: 2-1                                       --
│    │    └─DebertaV2Layer: 3-1                              5,662,144
├─Sequential: 1-3                                            --
│    └─BatchNorm1d: 2-2                                      3,840
│    └─Linear: 2-3                                           2,334,720
│    └─ReLU: 2-4                                             --
│    └─BatchNorm1d: 2-5                                      2,432
│    └─Linear: 2-6                                           1,478,656
│    └─BatchNorm1d: 2-7                                      2,432
├─Linear: 1-4                                                623,104
Total params: 10,109,248
Trainable params: 10,109,248
Non-tra

In [ ]:
trainer = pl.Trainer(
    logger=False, # CSVLogger('./'),
    enable_checkpointing=False,
    
    accelerator='gpu', 
    devices=1,
    profiler='advanced',
    precision=32,
    check_val_every_n_epoch=1,
    max_epochs=args.epochs
)

trainer.fit(model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type             | Params
--------------------------------------------------
0 | bn           | BatchNorm1d      | 1.9 K 
1 | deberta      | DebertaV2Encoder | 5.7 M 
2 | neck         | Sequential       | 3.8 M 
3 | output_layer | Linear           | 623 K 
--------------------------------------------------
10.1 M    Trainable params
0         Non-trainable params
10.1 M    Total params
40.437    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

D:\python_projects\ozon-hack-2023\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
D:\python_projects\ozon-hack-2023\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

In [ ]:
test_preds = trainer.predict(model, val_loader)

In [ ]:
roc_auc_score(val_pairs.target, np.concatenate([pred.numpy()[:, -1] for pred in test_preds]))

In [ ]:
# 0.8770786225758669

In [ ]:
train_dataset = ItemsDataset(train_pairs, train_data)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=args.batch_size,
    num_workers=0,
    drop_last=False,
    shuffle=False,
    pin_memory=True
)

In [ ]:
features = np.concatenate([pred.numpy() for pred in trainer.predict(model, train_loader)])

In [ ]:
val_features = np.concatenate([pred.numpy() for pred in trainer.predict(model, val_loader)])

In [ ]:
from catboost import CatBoostClassifier, Pool, cv

In [ ]:
train_pool = Pool(
    data=features,
    label=train_pairs.target,
)

val_pool = Pool(
    data=val_features,
    label=val_pairs.target,
)

In [ ]:
params = {
    'loss_function': 'Logloss',
    'custom_metric': ['AUC'],
    'task_type': 'CPU',
}

In [ ]:
model_cb = CatBoostClassifier(**params, random_seed=56)
model_cb.fit(train_pool, eval_set=val_pool, verbose=False, plot=True)

In [ ]:
np.max(model_cb.get_evals_result()['validation']['AUC'])

In [ ]:
# 0.8818639431064165

In [ ]:
cv_data = cv(
    params=params,
    pool=val_pool,
    fold_count=3,
    shuffle=True,
    partition_random_seed=0,
    plot=True,
    stratified=True,
    verbose=False
)

In [ ]:
max_iter = cv_data['test-AUC-mean'].argmax()
print(f"Best iteration: {max_iter}\nBest AUC: {round(cv_data.iloc[max_iter]['test-AUC-mean'], 4)}±{round(cv_data.iloc[max_iter]['test-AUC-std'], 4)}")

In [33]:
# 0.8915